In [ ]:
!git clone https://github.com/anagha1999/anlp-project/

fatal: destination path 'anlp-project' already exists and is not an empty directory.


In [ ]:
import requests
import os

urls = [
    "https://raw.githubusercontent.com/crvineeth97/kannada-stop-words/refs/heads/master/stop-words.txt"
]

for url in urls:
    filename = url.split('/')[-1]
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded: {filename}")

Downloaded: stop-words.txt


# Task
Use `pypdf` to extract text from specific pages of the following PDF files: pages 17-236 for "1-janapada-kathegalu.pdf" and pages 10-225 for "Satrii_Niiti_Kathegal'u_text.pdf". Then, load the stop words from "stop-words.txt", remove these words from the extracted texts, and save the cleaned content into a new directory named `kannada-pre-processed-texts` with the filenames `1-janapada-kathegalu.txt` and `Satrii_Niiti_Kathegal'u_text.txt`. Finally, summarize the extraction and cleaning process and confirm the paths of the saved files.

## Extract Text from PDFs

### Subtask:
Install `pypdf` and extract text from the specified page ranges of the two PDF files.


**Reasoning**:
Install the `pypdf` library to enable PDF text extraction.



In [ ]:
!pip install pypdf

**Reasoning**:
Extract text from the specified pages of the PDF files using `pypdf` and store the results in a dictionary, then print the first 500 characters of each extracted text.



In [13]:
import os
import requests
from pypdf import PdfReader

# Define filenames and page ranges
pdf_configs = [
    {'filename': '1-janapada-kathegalu.pdf', 'start': 17, 'end': 236},
    {'filename': "2-Satrii_Niiti_Kathegal'u_text.pdf", 'start': 10, 'end': 225}
]

# Directory where the repo was cloned
repo_dir = 'anlp-project/kannada/kannada-dataset'

extracted_texts = {}

for config in pdf_configs:
    filename = config['filename']
    start_page = config['start']
    end_page = config['end']

    # 1. Try to find the file in the repo directory
    file_path = os.path.join(repo_dir, filename)

    # 2. If not found, check current directory
    if not os.path.exists(file_path):
        file_path = filename

    # 3. If still invalid or corrupt, download it
    if not os.path.exists(file_path) or os.path.getsize(file_path) < 1000:
        print(f"File {filename} missing or corrupt. Downloading...")
        url = f"https://github.com/anagha1999/anlp-project/raw/main/kannada-dataset/{filename}"
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            file_path = filename
        else:
             print(f"Failed to download {filename}")

    print(f"Processing file: {file_path}")

    try:
        reader = PdfReader(file_path)
        text_content = []
        # Convert 1-based page numbers to 0-based indices
        # range(start, end) in python excludes end, so we need end_page (since we want up to end_page)
        # Instructions say: 1-janapada-kathegalu.pdf: pages 17 to 236.
        # Python index for page 17 is 16.
        # We want to include page 236. So range should go up to 236 (index 235).
        # range(16, 236) covers indices 16 to 235.
        for i in range(start_page - 1, end_page):
            if i < len(reader.pages):
                page = reader.pages[i]
                text_content.append(page.extract_text())

        extracted_texts[filename] = "\n".join(text_content)
        print(f"Successfully extracted text from {filename}")
    except Exception as e:
        print(f"Error extracting text from {file_path}: {e}")

# Print the first 500 characters of each extracted text
for filename, text in extracted_texts.items():
    print(f"\n--- Start of {filename} ---")
    print(text[:500])
    print(f"--- End of Preview ---")

Processing file: anlp-project/kannada/kannada-dataset/1-janapada-kathegalu.pdf
Successfully extracted text from 1-janapada-kathegalu.pdf
Processing file: anlp-project/kannada/kannada-dataset/2-Satrii_Niiti_Kathegal'u_text.pdf
Successfully extracted text from 2-Satrii_Niiti_Kathegal'u_text.pdf

--- Start of 1-janapada-kathegalu.pdf ---
೧. ಮುತುಗದ ಎಲೆ ಗೋತಾಯಿ 
ಒಂದ್‌ ಪಟ್ಟಣ. ಒಬ್ಬ ದೊಡ್ಡ ದೊರೆ. ಆ ದೊಡ್ಡ ದೊರೆಗೆ ಏಳ್‌ ಜನ ಮಕ್ಕಳು. ಆ 
ಏಳ್‌ ಜನ ಮಕ್ಕಳಿಗೂ ಚೆನ್ನಾಗಿ ಓದ್ದಿ, ಓದ್‌ ಬಂದೋರ್‌ಗೆ ಹಾಲು ತುಪ್ಪ ಅನ್ನ ಹಾಕಿ 
ಮಂಚದ ಮೇಲೆ ಕೂಡ್ರಿಸಿ ಚೆನ್ನಾಗಿ ಸಾಕೋರು. ಏಳು ಜನ ಮಕ್ಕಳೂವೆ ಹೆಣ್‌ ಮಕ್ಕಳೇ 
ಹುಟ್ಟಿದ್ದಾರಲ್ಲ ಅಂತ, ಆ ದೊರೆ ಒಂದಿನ ಹೆಣ್‌ಮಕ್ಕಳನ್ನೆಲ್ಲ ಕೇಳಿದನು, ನನ್ನಾರವ್ವ 
ಸಾಕೋರು ಅಂತ. ಆರು ಜನ ಮಕ್ಕಳೂವೆ ನಾನ್‌ ಸಾಕ್ತೀನಿ, ನಾನ್‌ ಸಾಕ್ತೀನಿಅಂತ ಅಂದ್ರು. 
ಆದರೆ ಕೊನೆ ಕಿರಿಮಗಳು ಮಾತ್ರ ನಾನ್‌ ಸಾಕಾಕಿಲ್ಲ ಅಂದ್ಲು. ಅಪ್ಪ ನಾವ್‌ ಸಾಕಾಕೆ ಆಗ್ಕದ? 
ಭಗವಂತ ನಮ್ಮನ್ನೆಲ್ಲ ಸಾಕ್‌ಬೇಕು. ಈ ಮಾತನ್ನೆ ಹೇಳ್ಕೊಂಡು ಬಂದ್ಲು. ಆಗ ಆ
--- End of Preview ---

--- Start of 2-Satrii_Niiti_Kathegal'u_text.pdf ---
ಸ್ತ್ರೀ ನೀತಿ ಕಥೆಗಳು 
ಪುರಾತನ ಕಾಲದಲ್ಲಿ ಗೋಪಾಲಸೆ ಟ್ಟ ಎಂಬೊಬ್ಬ ವ್ಯಾಪಾರಿಯು 
ಕಾಶಿ ರ ನಗರದಲ್ಲಿ ವಾ

**Reasoning**:
Load the stop words from the text file, create the output directory, clean the extracted texts by removing stop words, and save the cleaned texts to the specified files.



In [15]:
import os

# Load stop words
with open('stop-words.txt', 'r', encoding='utf-8') as f:
    stop_words = set(f.read().split())

# Create output directory
output_dir = 'kannada-pre-processed'
os.makedirs(output_dir, exist_ok=True)

cleaned_files = []

# Process each extracted text
for filename, text in extracted_texts.items():
    # Preserve structure by processing line by line
    cleaned_lines = []
    for line in text.splitlines():
        words = line.split()
        cleaned_words = [w for w in words if w not in stop_words]
        # Join words in the line and append to cleaned_lines
        # We keep empty lines to preserve vertical spacing if present
        cleaned_lines.append(" ".join(cleaned_words))

    cleaned_text = "\n".join(cleaned_lines)

    # Determine output filename (change extension to .txt)
    base_name = os.path.basename(filename)
    txt_filename = os.path.splitext(base_name)[0] + ".txt"
    output_path = os.path.join(output_dir, txt_filename)

    # Save the cleaned text
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(cleaned_text)

    cleaned_files.append(output_path)

print("Extraction and cleaning process completed (structure preserved).")
print("Saved files at:")
for path in cleaned_files:
    print(path)

Extraction and cleaning process completed (structure preserved).
Saved files at:
kannada-pre-processed/1-janapada-kathegalu.txt
kannada-pre-processed/2-Satrii_Niiti_Kathegal'u_text.txt


In [16]:
# Verify the structure of the cleaned text
output_path_check = 'kannada-pre-processed-texts/1-janapada-kathegalu.txt'

if os.path.exists(output_path_check):
    with open(output_path_check, 'r', encoding='utf-8') as f:
        print(f"--- Content Preview of {os.path.basename(output_path_check)} ---")
        # Read first 1000 characters to see structure
        print(f.read(1000))
        print("\n--- End of Preview ---")
else:
    print(f"File {output_path_check} not found. Make sure to run the processing cell first.")

--- Content Preview of 1-janapada-kathegalu.txt ---
೧. ಮುತುಗದ ಎಲೆ ಗೋತಾಯಿ ಒಂದ್‌ ಪಟ್ಟಣ. ಒಬ್ಬ ದೊಡ್ಡ ದೊರೆ. ದೊಡ್ಡ ದೊರೆಗೆ ಏಳ್‌ ಮಕ್ಕಳು. ಏಳ್‌ ಮಕ್ಕಳಿಗೂ ಚೆನ್ನಾಗಿ ಓದ್ದಿ, ಓದ್‌ ಬಂದೋರ್‌ಗೆ ಹಾಲು ತುಪ್ಪ ಅನ್ನ ಹಾಕಿ ಮಂಚದ ಕೂಡ್ರಿಸಿ ಚೆನ್ನಾಗಿ ಸಾಕೋರು. ಏಳು ಮಕ್ಕಳೂವೆ ಹೆಣ್‌ ಮಕ್ಕಳೇ ಹುಟ್ಟಿದ್ದಾರಲ್ಲ ಅಂತ, ದೊರೆ ಒಂದಿನ ಹೆಣ್‌ಮಕ್ಕಳನ್ನೆಲ್ಲ ಕೇಳಿದನು, ನನ್ನಾರವ್ವ ಸಾಕೋರು ಅಂತ. ಆರು ಮಕ್ಕಳೂವೆ ನಾನ್‌ ಸಾಕ್ತೀನಿ, ನಾನ್‌ ಸಾಕ್ತೀನಿಅಂತ ಅಂದ್ರು. ಕೊನೆ ಕಿರಿಮಗಳು ನಾನ್‌ ಸಾಕಾಕಿಲ್ಲ ಅಂದ್ಲು. ಅಪ್ಪ ನಾವ್‌ ಸಾಕಾಕೆ ಆಗ್ಕದ? ಭಗವಂತ ನಮ್ಮನ್ನೆಲ್ಲ ಸಾಕ್‌ಬೇಕು. ಮಾತನ್ನೆ ಹೇಳ್ಕೊಂಡು ಬಂದ್ಲು. ಮಂತ್ರಿನ, ರಾಜ ಕರೆದು ಹೆಣ್ಣು ಮಕ್ಕಳು ನಾವ್‌ ಸಾಕ್ತೀವಿ ನಾವ್‌ ಸಾಕ್ತೀವಿ, ಹೇಳ್ತಾರೆ - ಇವಳ್‌ ನಾನ್‌ ಸಾಕಾಕಿಲ್ಲ ಅಂತಾಳೆ. ಇವಳ್‌ನ ಯಾವದಾದರೊಂದು ಅಡವಿಗೆ ಕರಕೊಂಡೋಗಿ, ಇವಳನ್ನ ಕಡಿದು, ಇವಳ ರಕ್ತಾನ ಹಣೆಗೆ ತಂದು ಹಚ್ಚು ಮಂತ್ರೀನ ಕರೆದು ಹೇಳಿದ ರಾಜ. ಹಂಗಂತ ಅವನ ಕಿರಿಮಗಳನ್ನ ಕಳಿಸ್‌ಬಿಟ್ಟ. ಅವಳ ತಾಯಿಗೆ ತುಂಬ ದುಃಖವಾಯ್ತದೆ. ಆರು ಮಕ್ಕಳು ಮನೇಲೇ ಇದ್ದಾರೆ. ಇವಳನ್ನ ಕಾಡಿಗೆ ಕಳಿಸ್ತಾ ಇದ್ದಾರಲ್ಲ ಬಹಳ ಸಂಕಟ ಪಡ್ತಾಳೆ. ಮಂತ್ರಿ ಅಡವಿಗೆ ಕರೆದ್‌ಕೊಂಡ್‌ ಹೋಗ್ಬಿಟ್ಟ. ಅಡವಿಯಲ್ಲಿ ಏನ್‌ ಮನುಷ್ಯರ ಮುಖ ಇಲ್ಲ; ಅಂಥ ಅಡವಿಗೆ ಕರೆಕೊಂಡು ಬಂದ. ಹೆಣ್ಣು ಓದ್ಯೊಂಡ್‌ ಇದ್ದದ್ದು. ಅಲ್ಲೇ ನೆರೆದ್‌ಬಿಡ್ತು. ಸುಸ